# **Librerías**

In [12]:
#Realizamos todos los imports necesarios
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir("/content/drive/MyDrive/PLN/PracticaFinal")

import pandas as pd

import nltk
from nltk import WordPunctTokenizer

from nltk.corpus import stopwords
nltk.download('stopwords')
stops_words = stopwords.words('spanish')
#Añadimos los signos de puntuación más relevantes a las palabras vacías (lo de 'b' y 'r' es por un caracter residual que queda al hacer la lectura por bytes)
stops_words.extend(('.', ',', ':', ';', '?', '!', '-', '``', "''", '--', "'", "/", "(", ")", '"', "\\", "\\'", "...", 'b', 'r'))

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("spanish")

from sklearn import svm

from sklearn.feature_extraction.text import TfidfVectorizer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Funciones**



##   Función para leer los ficheros .tsv



In [13]:
def leerCorpus(nombre_fichero):
  #Leemos el fichero tsv de entrenamiento, poniendo como parámetros su nombre,
  #un delimitador de columnas y un encoding para que detecte el formato del texto
  df = pd.read_csv(nombre_fichero, delimiter='\t', encoding='unicode_escape')   #unicode_escape

  id = []
  comment = []
  label = []
  gender = []
  media = []

  #Pasamos la información de las columnas a listas
  for i in range (len(df.index)):
    #print(df.iloc[:, 0][i])
    id.append(df.iloc[:, 0][i])
    correccion = correccionCodificacion(df.iloc[:, 1][i])
    comment.append(correccion)
    label.append(df.iloc[:, 2][i])
    gender.append(df.iloc[:, 3][i])
    media.append(df.iloc[:, 4][i])

  #Creamos un diccionario con los textos y los géneros
  doc = {
    "id": id,
    "comment": comment,
    "label": label,
    "gender": gender,
    "media":media
  }

  return doc 





##   Función para procesar el texto





In [14]:
def procesarTexto(texto):

  #Tokenizamos (he elegido este tokenizador por la separación qque hace de la puntuación)
  tokens = WordPunctTokenizer().tokenize(texto)

  #Cambiamos todos los tokens a minúsculas
  tokens_minus = []
  for word in tokens:
    tokens_minus.append(word.lower())

  #Eliminamos palabras vacías
  tokens_filtrados = [word for word in tokens_minus if word not in stops_words]

  #Reducimos las palabras a su raiz
  tokens_reducidos = ""
  for word in tokens_filtrados:
    tokens_reducidos = tokens_reducidos + stemmer.stem(word) + " "
    #tokens_reducidos.append(stemmer.stem(word))

  return tokens_reducidos

#texto_procesado = procesarTexto(doc["comment"][12]) 





##   Función para corregir la mala lectura de los ficheros




In [15]:
def correccionCodificacion(cadena):
  corregida = ""

  for word in cadena.split(' '):
    if 'Ã­' in word:
      aux = word.replace('Ã', 'í')
      corregida += aux + " "
    elif 'Ã¡' in word:
      aux = word.replace('Ã¡', 'á')
      corregida += aux + " "
    elif 'Ãº' in word:
      aux = word.replace('Ãº', 'ú')
      corregida += aux + " "
    elif 'Ã©' in word:
      aux = word.replace('Ã©', 'é')
      corregida += aux + " "
    elif 'Ã³' in word:
      aux = word.replace('Ã³', 'ó')
      corregida += aux + " "
    elif 'Ã±' in word:
      aux = word.replace('Ã±', 'ñ')
      corregida += aux + " "
    else:
      corregida += word + " "

  return corregida



# **Código principal**


##  Leemos y procesamos el fichero de entrenamiento



In [16]:
#Leemos el corpus
doc = leerCorpus("train.tsv")

#Procesamos los textos leidos del corpus
textos = []
for i in doc["comment"]:
  texto_procesado = procesarTexto(str(i))
  textos.append(texto_procesado)

#Actualizamos los textos en el diccionario
doc["comment_procesados"] = textos




##   Leemos el fichero para la evaluación



In [17]:
#Leemos el corpus test para predecir el genero de la persona que los ha escrito
fichero_test = leerCorpus("test.tsv")

#Procesamos los textos leidos del corpus
textos_test = []
for i in fichero_test["comment"]:
  texto_procesado = procesarTexto(str(i))
  textos_test.append(texto_procesado)

#Actualizamos los textos en el diccionario
fichero_test["comment_procesados"] = textos_test



##   Entrenamos el modelo



In [18]:
#Entrenamos al modelo
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(doc["comment_procesados"])

model = svm.SVC()
model.fit(X, doc["label"])

SVC()

## Creamos fichero de salida

In [19]:
#Creamos el fichero y preparamos la primera fila (títulos)
f = open("prediction_test.tsv", "w")
f.write("comment_id\tpred")

15

## Hacemos predicción y anotamos resultados en fichero de salida

In [20]:
cont = 0

#Realizamos la predicción y la anotamos en un fichero
for comentario in fichero_test["comment_procesados"]:
  text = [comentario]

  Y = vectorizer.transform(text)
  prediction = model.predict(Y)

  #Eliminamos los caracteres que no nos interesan de la predicción
  caracteres = "[']"
  prediccion = ''.join( x for x in str(prediction) if x not in caracteres)

  #Escribimos en el fichero
  f.write("\n" + str(fichero_test["id"][cont]) + "\t" + prediccion)

  cont += 1

f.close()



##   Evaluamos el sistema



In [21]:
#Se ejecuta el script de prueba
%run -i scorer.py gold_label_test.tsv prediction_test.tsv

GOLD_FILE: gold_label_test.tsv
PREDICTION_FILE: prediction_test.tsv


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

{'maf': 0.7041676707731491, 'map': 0.8848711769084219, 'mar': 0.6612039939286498, 'mif': 0.8679259150374835, 'mip': 0.8679259150374835, 'mir': 0.8679259150374835, 'avgf': 0.8417355962126334, 'avgp': 0.8725987780879091, 'avgr': 0.8679259150374835}
